<a href="https://colab.research.google.com/github/mattneate09/UTS_ML2019_ID12569998/blob/master/Ass2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Mon Sep 16 17:39:34 2019

@author: Matt
"""

#%%
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 
import random



#%%

Train_dir = "D:\\UNICODE\\IDC"

Train_class0 = ["D:\\UNICODE\\IDC\\class0\\{}".format(i) for i in os.listdir(Train_dir+"\\class0")]
Train_class1 = ["D:\\UNICODE\\IDC\\class1\\{}".format(i) for i in os.listdir(Train_dir+"\\class1")]


Train_imgs = Train_class1[:10000]+ Train_class0[:10000]  
random.shuffle(Train_imgs)

del Train_class1
del Train_class0

print(len(Train_imgs))



#%%
x=1
import matplotlib.image as mpimg
for ima in Train_imgs[0:3]:
    img = mpimg.imread(ima)
    imgplot = plt.imshow(img)
    plt.show() 
    ##print(x)
    x=x+1

#%%
nrows = 50
ncolumns =50
channels =1 

def ImageProcessandRead(list_of_images):

    X = []
    y = []

    for images in list_of_images:   
        X.append(cv2.resize(cv2.imread(images,cv2.IMREAD_COLOR),(nrows,ncolumns), interpolation=cv2.INTER_CUBIC))  
        if 'class1' in images:
            y.append(1)
        elif 'class0'in images:
            y.append(0)  

    return X,y

X, y = ImageProcessandRead(Train_imgs)

#%%
#print(y)
del Train_imgs

#%%
import seaborn as sns

X= np.array(X)
y = np.array(y)
print("follwing two varibles")
print(X.shape)
print(y.shape)
sns.countplot(y)
plt.title('Labels for Class 1 and 0')

print("shape of Training images is ",X.shape)
print("shape of Labels is",y.shape)


#%%
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.20, random_state =2)

del X
del y 
#%%
ntrain = len(X_train)
nval = len(X_val)

batch_size = 800
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img

model = models.Sequential()
model.add(layers.Conv2D(2,(3,3),activation = 'relu',input_shape=(50,50,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(4,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(8,(3,3),activation='relu'))
model.add(layers.Dropout(0.4))
model.add(layers.Conv2D(16,(3,3),activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(32,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(1,activation = 'sigmoid'))





#%%
model.summary()

#%%
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(lr=0.0001), metrics=['acc'])

train_datagen = ImageDataGenerator(rescale=1./200,rotation_range=40,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,)

val_datagen = ImageDataGenerator(rescale=1./200)

train_generator = train_datagen.flow(X_train,y_train,batch_size=batch_size)
val_generator = train_datagen.flow(X_val,y_val,batch_size=batch_size)

history = model.fit_generator(train_generator,steps_per_epoch=ntrain // batch_size,epochs = 1000,validation_data = val_generator, validation_steps = nval//batch_size)

model.save_weights('model_weights.h5')
model.save('model_keras.h5')

#%%

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1,len(acc)+1)

#%%
plt.clf()
plt.plot(epochs,acc,'b',label = 'Training accuracy')
plt.plot(epochs,val_acc,'r',label = 'Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

plt.plot(epochs,loss,'g',label = 'Training loss')
plt.plot(epochs,val_loss,'y',label = 'Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

#%%






